# Cine21 Crawling 1
네이버 영화 아카이브에서 최근 3년(2019-2021)에 개봉한 영화의 리스트를 가져와 씨네21에서 매칭되는 영화 찾기

## 1. Import libraries

In [1]:
import time
import pandas as pd
import csv

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

## 2. Connect to Cine21 website

In [2]:
start_time = time.time()

from selenium import webdriver

#C:\Users\Danbee\Desktop\sample-project\data
driver = webdriver.Chrome("C:/Users/Danbee/Desktop/sample-project/data/chromedriver")
#driver = webdriver.Chrome("C:\Users\Danbee\Desktop\sample-project\data\chromedriver")

# 씨네21 영화 검색 페이지 접속 (''검색 결과)
driver.get("http://www.cine21.com/")

print("--- %s seconds ---" % (time.time() - start_time))

--- 20.026793003082275 seconds ---


## 3. Prepare Data
네이버 3년치 데이터 가져온 뒤 취합

In [3]:
naver_df_2019 = pd.read_csv('C:/Users/Danbee/Desktop/luckyseven/naver_data/naver_2019_poster.csv', encoding='UTF8')
naver_df_2020 = pd.read_csv('C:/Users/Danbee/Desktop/luckyseven/naver_data/naver_2020_poster.csv', encoding='UTF8')
naver_df_2021 = pd.read_csv('C:/Users/Danbee/Desktop/luckyseven/naver_data/naver_2021_poster.csv', encoding='UTF8')

In [4]:
print(len(naver_df_2019)) # 722 vs 850
print(len(naver_df_2020)) # 647 vs 730
print(len(naver_df_2021)) # 549 vs 728

# 누락된 데이터 수
print("2019년에서 누락된 데이터 수:", 850-722)
print("2020년에서 누락된 데이터 수:", 730-647)
print("2021년에서 누락된 데이터 수:", 728-549)

# 3년 전체
print("2019~2021년에서 누락된 데이터 수:", (850+730+728)-(722+647+549))

722
647
549
2019년에서 누락된 데이터 수: 128
2020년에서 누락된 데이터 수: 83
2021년에서 누락된 데이터 수: 179
2019~2021년에서 누락된 데이터 수: 390


In [5]:
# 모아온 3년 전체 데이터 수 = 722+647+549 = 1918
df = pd.concat([naver_df_2019, naver_df_2020, naver_df_2021], ignore_index=True)
df

,title,genre,nation,running,release,director,actor,grade,story,poster
0,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...
1,1 더하기 3,멜로/로맨스,홍콩,101분,2019 .12.12 개봉,프룻 첸,증미혜자,청소년 관람불가,"멈추지 않는 성욕과 사랑하는 세 명의 남편을 위해,\n 선상에서 밤낮 없이 몸을 파...",https://movie-phinf.pstatic.net/20191204_25/15...
2,10 미니츠 곤,"액션, 범죄",캐나다,96분,2019 .11.21 개봉,브라이언 A 밀러,"브루스 윌리스, 마이클 치클리스",15세 관람가,동물적인 직감과 1초의 오차도 용납하지 않는 설계자 ‘렉스’\n 철저한 계획 하에 ...,https://movie-phinf.pstatic.net/20191104_204/1...
3,100일 동안 100가지로 100퍼센트 행복찾기,코미디,독일,111분,2019 .09.12 개봉,플로리안 데이비드 핏츠,"플로리안 데이비드 핏츠(폴), 마치아스 슈와바이어퍼(토니)",15세 관람가,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,\n 사는 집도 아랫집, 윗집으...",https://movie-phinf.pstatic.net/20190823_42/15...
4,10년,드라마,일본,99분,2019 .12.12 개봉,"하야카와 치에, 키노시타 유스케, 츠노 메구미","스기사키 하나, 쿠니무라 준, 이케와키 치즈루",전체 관람가,"안락사, 인공지능과 완전히 통제된 교육,\n 디지털 유산과 알 권리, 환경오염, 전...",https://movie-phinf.pstatic.net/20191120_7/157...
...,...,...,...,...,...,...,...,...,...,...
1913,해변의 에트랑제,"애니메이션, 멜로/로맨스",일본,59분,"2021 .09.09 재개봉, 2021 .02.18 개봉",오오하시 아키요,"심규혁(하시모토 슌), 이경태(치바나 미오), 무라타 타이시",15세 관람가,"오키나와 외딴섬, 해변 벤치에 혼자 멈춰있는 소년 ‘미오’. 그런 미오가 몹시 신경...",https://movie-phinf.pstatic.net/20210125_20/16...
1914,헬렌: 내 영혼의 자화상,드라마,핀란드,121분,2021 .02.25 개봉,안티 조키넨,로라 비른,12세 관람가,“조건이 있어요. 당신을 그리고 싶어요”\n \n 10년 넘게 예술계를 벗어나\n ...,https://movie-phinf.pstatic.net/20210205_206/1...
1915,호라이즌 라인,스릴러,미국,92분,2021 .02.10 개봉,마이클 마키마인,"알렉산더 드레이먼(잭슨), 앨리슨 윌리암스(사라)",12세 관람가,에메랄드 빛 해안을 낀 열대섬으로 향하던 ‘사라’\n 이륙 후 두근거림도 잠시 멀쩡...,https://movie-phinf.pstatic.net/20210125_149/1...
1916,호스트: 접속금지,공포,영국,59분,2021 .07.21 개봉,롭 새비지,"헤일리 비숍(헤일리), 젬마 무어(젬마), 엠마 루이즈 웨브(엠마)",15세 관람가,"팬데믹, 락다운과 함께 자가격리를 시작한 ‘헤일리’와 친구들.\n ‘줌’을 통해 랜...",https://movie-phinf.pstatic.net/20210708_28/16...


## 4. find the match and get urls
네이버 영화 리스트를 받아서 씨네21에서 title로 검색하여 나온 결과 페이지의 1페이지를 차례로 보면서
title과 director가 매칭되면 해당 영화의 상세페이지로 가는 url을 리턴한다.

In [6]:
def find_matching_movie(title_director_list):
    
    title = title_director_list[0]
    director = title_director_list[1]
    
    # 검색창: input#search_q.input_search
    search_box = driver.find_element_by_css_selector("input#search_q.input_search")
    # 검색버튼: a.btn_search
    search_btn = driver.find_element_by_css_selector("#total_search_form > fieldset > a")
    
#     time.sleep(2)
    
    # 홈페이지에서 제목으로 검색
    search_box.send_keys(title)
    search_btn.click()
    
#     time.sleep(2)

    # 영화 탭 버튼 눌러서 영화에 대한 검색 결과만 보기
    movie_tab_btn = driver.find_element_by_css_selector("#snb_t > li:nth-child(2) > a")
    movie_tab_btn.click()
    
    time.sleep(2)

    # 일단 해당 영화 제목으로 검색해서 검색 결과가 있는 지 확인
    try:
        # 결과가 하나라도 있는 경우
        # 검색 결과 수 보기
        num_of_movies = driver.find_element_by_css_selector("#content > div.culm2_area > div.culm2_l > h2 > span").text
        num_of_movies = int(num_of_movies.replace('영화', '').replace('(', '').replace(')', '').replace(' ', ''))
#         print(f'씨네21에서 {title}에 대해 찾은 검색 결과는 {num_of_movies}건 입니다.')

#         time.sleep(2)
        
        # 영화 검색 결과 페이지에서
        # 일단 1페이지에 내가 찾는 있다고 가정
        # 영화 제목, 감독 일치하면 해당 페이지로 들어가서 정보 가져오기
        
        for i in range(1, 11):

            titles = driver.find_elements_by_css_selector(f"#content > div.culm2_area > div.culm2_l > ul.mov_list > li:nth-child({i}) > p.name > a")
            
            try:
                href = titles[0].get_attribute('href')
                directors = driver.find_elements_by_css_selector(f"#content > div.culm2_area > div.culm2_l > ul.mov_list > li:nth-child({i}) > p:nth-child(4) > a:nth-child(2)")
                
#                 print('제목:', title, titles[0].text)
#                 print('감독:', director, directors[0].text)

    #             time.sleep(2)

                if (title == titles[0].text) and (director.replace('.', '') == directors[0].text.replace('.','')):
                    return href

                else:
#                     print(f'{title}와/과 제목과 감독이 일치하는 영화가 이 페이지에 없습니다.')
                    return('매칭 영화 없음')
    #                 continue
    
            except IndexError:
                pass
#                 print(f"{title}에 대한 감독 정보가 없습니다")
    
    except NoSuchElementException:
#         print(f"{title}에 대한 검색 결과가 없습니다.")
        return('검색 결과 없음')

## 네이버 영화 데이터에서 제목과 감독 이름 매칭해서 씨네21 사이트에서 매칭되는 영화 찾아 해당 영화의 상세페이지로 연결되는 url 리스트 얻어오기

In [11]:
naver_list = list(map(list, zip(df['title'], df['director'])))
len(naver_list)

1918

In [ ]:
urls_from_cine = []

In [17]:
start_time = time.time()

for i in naver_list:
    print(f"현재 {naver_list.index(i)+1}번째 영화를 찾고 있습니다.")
    result = find_matching_movie(i)
    urls_from_cine.append(result)
    
print("--- %s seconds ---" % (time.time() - start_time))

현재 1175번째 영화를 찾고 있습니다.
현재 1176번째 영화를 찾고 있습니다.
현재 1177번째 영화를 찾고 있습니다.
현재 1178번째 영화를 찾고 있습니다.
현재 1179번째 영화를 찾고 있습니다.
현재 1180번째 영화를 찾고 있습니다.
현재 1181번째 영화를 찾고 있습니다.
현재 1182번째 영화를 찾고 있습니다.
현재 1183번째 영화를 찾고 있습니다.
현재 1184번째 영화를 찾고 있습니다.
현재 1185번째 영화를 찾고 있습니다.
현재 1186번째 영화를 찾고 있습니다.
현재 1187번째 영화를 찾고 있습니다.
현재 1188번째 영화를 찾고 있습니다.
현재 1189번째 영화를 찾고 있습니다.
현재 1190번째 영화를 찾고 있습니다.
현재 1191번째 영화를 찾고 있습니다.
현재 1192번째 영화를 찾고 있습니다.
현재 1193번째 영화를 찾고 있습니다.
현재 1194번째 영화를 찾고 있습니다.
현재 1195번째 영화를 찾고 있습니다.
현재 1196번째 영화를 찾고 있습니다.
현재 1197번째 영화를 찾고 있습니다.
현재 1198번째 영화를 찾고 있습니다.
현재 1199번째 영화를 찾고 있습니다.
현재 1200번째 영화를 찾고 있습니다.
현재 1201번째 영화를 찾고 있습니다.
현재 1202번째 영화를 찾고 있습니다.
현재 1203번째 영화를 찾고 있습니다.
현재 1204번째 영화를 찾고 있습니다.
현재 1205번째 영화를 찾고 있습니다.
현재 1206번째 영화를 찾고 있습니다.
현재 1207번째 영화를 찾고 있습니다.
현재 1208번째 영화를 찾고 있습니다.
현재 1209번째 영화를 찾고 있습니다.
현재 1210번째 영화를 찾고 있습니다.
현재 1211번째 영화를 찾고 있습니다.
현재 1212번째 영화를 찾고 있습니다.
현재 1213번째 영화를 찾고 있습니다.
현재 1214번째 영화를 찾고 있습니다.
현재 1215번째 영화를 찾고 있습니다.
현재 1216번째 영화를 찾고 있습니다.
현재 1217번째 영화를 찾고 있습니다.
현재 1218번째 영

In [124]:
urls_from_cine.to_csv("C:/Users/Danbee/Desktop/sample-project/data/cine21_urls_df.csv", index=False, sep="|", na_rep="NaN", encoding='utf-8-sig')

In [133]:
with open('./cine21_urls_df.csv', encoding='utf-8') as file:
    csv.reader(file)
    cine21_urls_df = pd.read_csv(file)

In [134]:
cine21_urls_df

,url
0,0
1,http://www.cine21.com/movie/info/?movie_id=52988
2,검색 결과 없음
3,http://www.cine21.com/movie/info/?movie_id=55539
4,http://www.cine21.com/movie/info/?movie_id=55031
...,...
1911,http://www.cine21.com/movie/info/?movie_id=57513
1912,매칭 영화 없음
1913,http://www.cine21.com/movie/info/?movie_id=57226
1914,http://www.cine21.com/movie/info/?movie_id=57001


In [136]:
cine21_urls_df
print(len(cine21_urls_df))

cine21_urls_df = cine21_urls_df.drop([0])
print(len(cine21_urls_df))

1916
1915


In [137]:
cine21_urls_list = cine21_urls_df.url.values.tolist()
cine21_urls_list
len(cine21_urls_list)

1915